<a href="https://colab.research.google.com/github/aymanmostafa11/Andromeda/blob/main/NasaSpaceApps'22.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Can AI preserve our science legacy

In [1]:
import requests
import json

In [2]:
api_root = 'https://ntrs.nasa.gov'

In [3]:
def getDocsLinks(count=100):
  offset = 0
  url = api_root + f'/api/citations/search?center=CDMS&page.size=100&offset={offset}'

  download_urls = []
  while len(download_urls) < count:
    resp = requests.get(url).json()
    #print(resp['results'][-1]['downloads'][0]['links']['fulltext'])
    downloadable_files = [ ( i['id'] ,i['downloads'][0]['links']['fulltext'] )
                          for i in resp['results'] 
                          if len(i['downloads']) > 0]
    download_urls.extend(downloadable_files)
    offset += 100

  return download_urls[:count]


In [ ]:
docs = getDocsLinks(10)

In [ ]:
docs

In [ ]:
texts = [ (i[0], requests.get(api_root + i[1]).content) for i in docs ]

In [ ]:
import nltk
from nltk.stem import WordNetLemmatizer
import gensim

In [ ]:
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


True

In [ ]:
words = gensim.utils.simple_preprocess(texts[0][1])

In [ ]:
results = [WordNetLemmatizer().lemmatize(i,'v')
           for i in words 
           if len(i) > 3 and 
           i not in gensim.parsing.preprocessing.STOPWORDS]

In [ ]:
results

In [ ]:
dic = gensim.corpora.Dictionary([results,])

In [ ]:
bow = dic.doc2bow(results)

In [ ]:
print(dic.items())

ItemsView(<gensim.corpora.dictionary.Dictionary object at 0x7f6bf1334d50>)


In [ ]:
[[(dic[id], count) for id, count in line] for line in bow]

TypeError: ignored

In [ ]:
[ for i in bow]

AttributeError: ignored

# Summarization

In [ ]:
resp = requests.get(api_root + '/api/citations/19810002806').json()

In [ ]:
doc = str(requests.get(api_root + resp['downloads'][0]['links']['fulltext']).content)

In [ ]:
import gensim
from gensim.summarization import summarize

In [ ]:
print(summarize(doc,0.01), sep='\n\n')

Type of Report and Period covered\n\nTRW Defense and Space Systems Group\nPower Conversion Electronics Department\nOne Space Park\nRedondo Beach, California\t 90278\t\n_\n12, Sponsoring Ap ncy Nema and Address\n\nFinal TECH.\nJune 1976- Jan. 1980\n14, Sponsoring Apmcy code\n\nNASA Lewis Research Center\n21000 Brookpark Road\nC leveland, Ohio\t 44135\n\n15, Supplementary Notes\n\nNASA Technical Monitor:\t\n\n18,\n\n4\n\nJoseph Kolecki\n\nAbstract\nThree basic switchin g regulators:\t\n\nbuck, boost, and buck/boost, employing a Standardized\n\nControl Module (SCM) were characterized by a common small signal block diagram in Volume I of\n\nthe report, "Application Handbook for a Standardized Control Module for DC-DC Converters,"\nEmploying the unified model, regulator performance such as stability, aud\'iosusceptibility,\n\noutput impedance, and step-load transient were analyzed and key performance indices were\nexpressed in simple analytical forms,\t It has been demonstrated that the per

In [ ]:
pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.9 MB 16.1 MB/s 
     |████████████████████████████████| 163 kB 70.4 MB/s 
     |████████████████████████████████| 6.6 MB 51.4 MB/s 


In [ ]:
pip install sentencepiece

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.3 MB 12.4 MB/s 


In [ ]:
from transformers import T5Tokenizer, T5Config, T5ForConditionalGeneration

In [ ]:
model = T5ForConditionalGeneration.from_pretrained('t5-small')
tokenizer = T5Tokenizer.from_pretrained('t5-small')

TypeError: ignored

In [ ]:
T5Tokenizer.from_

transformers.utils.dummy_sentencepiece_objects.T5Tokenizer